The point of this project was to utilize more than just sequence data for protein classification. Standard apporaches use reccurrent neural networks on protein sequences to predict protein classification, but we went about trying to see if we could use other easily attainable features to aid with prediction. 

| Experimentation No | Input Data | Model |
| --- | ----------- | ---------- |
| 1 | Protein Sequence Character Embeddings and Numerical Data | Multi-Input LSTM Concatenated Model |
| 2 | Protein Sequence Frequency Vector Embedding and Numerical Data | Multi-Input LSTM Concatenated Model |
| 3 | Protein Sequence Character Embeddings and Numerical Data | Multi-Input LSTM On Top of CNN Concatenated Model |
| 4 | Protein Sequence Character Embeddings and Numerical Data | Multi-Input Deep Dense LSTM On Top of CNN with Concatenated Model |
| FINAL | Merged Protein Sequence Data and Numerical Data into Single Feature | Single-Input LSTM On Top of CNN Model |

**Please Note:**  *For convenience's sake, this notebook only includes the final approach*

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. EDA and Data Preprocessing

In [ ]:
seq = pd.read_csv('/kaggle/input/protein-data-set/pdb_data_seq.csv')
seq

In [ ]:
NoDups = pd.read_csv('/kaggle/input/protein-data-set/pdb_data_no_dups.csv')
NoDups

In [ ]:
df = NoDups.merge(seq,how='inner',on='structureId').drop_duplicates(['structureId'])
df

In [ ]:
print(len(df))

In [ ]:
df.isnull().sum()

In [ ]:
len(df)

In [ ]:
df=df.dropna()
len(df)

In [ ]:
df = df[df['macromoleculeType_x'] == 'Protein']
print(len(df))
print(df)

In [ ]:
df=df.reset_index()
df

In [ ]:
df=df.drop(['index'],axis=1)
df

In [ ]:
count=0
for c in df.classification.values:
    if c=='OXYGEN TRANSPORT':
        count+=1
print(count)

In [ ]:
print(df.residueCount_x.quantile(0.9))
df.residueCount_x.describe()

In [ ]:
df.columns

In [ ]:
df = df.loc[df.residueCount_x<1200]
df.residueCount_x.describe()

In [ ]:
df.head()

In [ ]:
df['merge_col'] = list(map(lambda i : str(i['sequence']) + ' ' + str(i['residueCount_y'])+" "+str(i['phValue'])+" "+str(i['densityPercentSol'])+" "+str(i['macromoleculeType_y']) + " " + str(i['residueCount_x']) + " " + str(i['resolution']) + " " + str(i['crystallizationTempK']) + " " + str(i['chainId']) ,df.to_dict(orient="records") ))

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
from collections import Counter

In [ ]:
uniqueC=0
uniqueL=[]
for c in df.classification:
    if c not in uniqueL:
        uniqueL.append(c)
        uniqueC+=1
print(uniqueC)

In [ ]:
df.classification.nunique()

In [ ]:
# count numbers of instances per class
cnt = Counter(df.classification)
# select only K most common classes! - was 10 by default
top_classes = 10
# sort classes
sorted_classes = cnt.most_common()[:top_classes]
classes = [c[0] for c in sorted_classes]
counts = [c[1] for c in sorted_classes]
print("at least " + str(counts[-1]) + " instances per class")

# apply to dataframe
print(str(df.shape[0]) + " instances before")
df = df[[c in classes for c in df.classification]]
print(str(df.shape[0]) + " instances after")

seqs = df['merge_col']
lengths = [len(s) for s in seqs]

In [ ]:
len(df.classification.value_counts())

In [ ]:
df.classification.value_counts()

In [ ]:
df.head()

In [ ]:
df.columns

# Data Preparation

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y = lb.fit_transform(df.classification)

In [ ]:
X=df.drop('classification',axis=1)

In [ ]:
df.iloc[0]['merge_col']

In [ ]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

max_length = max(lengths)

tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(seqs)

X_seq = tokenizer.texts_to_sequences(seqs)
X_seq = sequence.pad_sequences(X_seq, maxlen=max_length)

In [ ]:
df.info()

In [ ]:
from tensorflow.keras.models import Sequential
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.merge import Concatenate
from tensorflow.keras.layers import Dense,Dot,Conv1D,MaxPooling1D,Activation,Dropout,LSTM,Flatten,GlobalMaxPooling1D,Input

In [ ]:
print(tokenizer.__dict__)

In [ ]:
tokenizer.word_index

In [ ]:
# def vector_based_on_freq(sequence):
#     vector = []
#     for i in tokenizer.word_index.keys():
#         i=i.upper()
#         vector.append(sequence.count(i)/len(sequence))
    
#     return(vector)

In [ ]:
tokenizer.word_index.keys()

In [ ]:
# vector_based_on_freq(df.iloc[0]['sequence'])

In [ ]:
# df['frequency_vector'] = df['sequence'].apply(lambda i : vector_based_on_freq(i))

In [ ]:
# X_rest = df.drop(['sequence','classification','macromoleculeType_x','structureId',
#                  'experimentalTechnique','crystallizationMethod','pdbxDetails',
#                  'chainId','macromoleculeType_y','frequency_vector'],axis=1)

In [ ]:
# X_rest = df.drop(['sequence','classification','macromoleculeType_x','structureId',
#                  'experimentalTechnique','crystallizationMethod','pdbxDetails',
#                  'chainId','macromoleculeType_y','publicationYear'],axis=1)

In [ ]:
# X_rest.head()

In [ ]:
# X_seq = df['frequency_vector']

# Final Model

In [ ]:
input_1 = Input(shape=(max_length,)) #sequence

In [ ]:
embedding_layer = Embedding(len(tokenizer.word_index)+1, 8)(input_1)
CNN_Layer_pre = Conv1D(filters=64,kernel_size=6,padding='same',activation='relu')(embedding_layer)
CNN_layer = Conv1D(filters=32,kernel_size=3,padding='same',activation='relu')(CNN_Layer_pre)
MaxPooling_layer = MaxPooling1D(pool_size=2)(CNN_layer)
LSTM_Layer_1 = LSTM(150)(MaxPooling_layer)
#concat_layer = Concatenate()([LSTM_Layer_1, input_2])
#dense_Concat = Dense(100,activation='relu')(concat_layer)
#dense_Concat_2 = Dense(100,activation='relu')(dense_Concat)

In [ ]:
# dense_layer_3 = Dense(10, activation='relu')(concat_layer)
output = Dense(top_classes, activation='softmax')(LSTM_Layer_1)
model = Model(inputs=input_1, outputs=output)

# A Prototype Model 

In [ ]:
# input_1 = Input(shape=(25,)) #sequence

In [ ]:
# input_2 = Input(shape=(len(X_rest.columns),)) #rest

In [ ]:
# concat_layer = Concatenate()([input_1, input_2])
# dense_layer_3 = Dense(10, activation='relu')(concat_layer)
# output = Dense(top_classes, activation='softmax')(dense_layer_3)
# model = Model(inputs=[input_1, input_2], outputs=output)

# Training and Testing Model

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])
print(model.summary())

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot3.png', show_shapes=True, show_layer_names=True)

In [ ]:
X_seq = np.array(X_seq)

In [ ]:
X_seq = np.array(list(X_seq))

In [ ]:
msk = np.random.rand(len(df)) < 0.8

In [ ]:
X_seq_train = X_seq[msk]
# X_rest_train = X_rest[msk]
y_train = y[msk]

X_seq_test = X_seq[~msk]
# X_rest_test = X_rest[~msk]
y_test = y[~msk]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
best_model_save = ModelCheckpoint(
    '/kaggle/working/best_model.h5', monitor='val_acc', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto'
)


In [ ]:
history = model.fit(x=X_seq_train, y=y_train, batch_size=128, epochs=15, verbose=1, validation_split=0.2,callbacks=[best_model_save])

In [ ]:
import tensorflow as tf
best_model = tf.keras.models.load_model('/kaggle/working/best_model.h5')

In [ ]:
from sklearn.metrics import confusion_matrix
y_test_pred = best_model.predict(X_seq_test)
confusionMatrix = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(y_test_pred,axis=1))

In [ ]:
# Plot normalized confusion matrix
confusionMatrix = confusionMatrix.astype('float') / confusionMatrix.sum(axis=1)[:, np.newaxis]
np.set_printoptions(precision=2)
plt.figure(figsize=(10,10))
plt.imshow(confusionMatrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(top_classes)
plt.xticks(tick_marks, sorted_classes, rotation=90)
plt.yticks(tick_marks, sorted_classes)
#for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#    plt.text(j, i, format(cm[i, j], '.2f'), horizontalalignment="center", color="white" if cm[i, j] > cm.max() / 2. else "black")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

confusionMatrix

In [ ]:
#References
#https://towardsdatascience.com/a-comprehensive-guide-to-correlational-neural-network-with-keras-3f7886028e4a
#https://snap.stanford.edu/snappy/doc/reference/multimodal.html
#https://openaccess.thecvf.com/content_cvpr_2017/papers/Chen_AMC_Attention_guided_CVPR_2017_paper.pdf

# References
https://towardsdatascience.com/a-comprehensive-guide-to-correlational-neural-network-with-keras-3f7886028e4a

https://snap.stanford.edu/snappy/doc/reference/multimodal.html

https://openaccess.thecvf.com/content_cvpr_2017/papers/Chen_AMC_Attention_guided_CVPR_2017_paper.pdf